In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPool2D  # Import Conv2D
from tensorflow.keras import models, layers, datasets, utils
import matplotlib.pyplot as plt

In [5]:
IMG_ROOT_DIR = "C:/Faster_Code/ASSN_7_INT_SYSTEMS/micro_QR_codes"
# Global params for loading the images
batch_size = 8
img_height = 370
img_width = 370

In [10]:
loadel_model = tf.keras.models.load_model('WholeSet2Epoch.keras')

qr_train, qr_test = utils.image_dataset_from_directory(
    IMG_ROOT_DIR,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Normalization / Scaling of colors
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = qr_train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

# Cache datasets for performance
AUTOTUNE = tf.data.AUTOTUNE

qr_train = qr_train.cache().prefetch(buffer_size=AUTOTUNE)
qr_test = qr_test.cache().prefetch(buffer_size=AUTOTUNE)



ValueError: File not found: filepath=WholeSet2Epoch.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
evaluation = loaded_model.evaluate(qr_test)
print("Accuracy of testing: {}".format(evaluation[1]))